# S023-dataprocessing Job
### 초기데이터 적재후에 DW 마트를 생성하는 부분
### 기존 Airflow의 다음 부분이 해당
```python
get_raw_3 >> prep_hospital >> target_revision >> union_hospital >> multiplication >> merge_raw
get_raw_3 >> prep_pharmacy >> multiplication_p >> merge_raw
get_raw_3 >> prep_dentist >> merge_raw
```

In [2]:
%%configure 
{
    "--datalake-formats": "iceberg",
    "--conf": "spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.type=glue --conf spark.sql.defaultCatalog=glue_catalog"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 
The following configurations have been updated: {'--datalake-formats': 'iceberg', '--conf': 'spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.type=glue --conf spark.sql.defaultCatalog=glue_catalog'}


In [1]:
%idle_timeout 60
%glue_version 5.0
%worker_type G.1X
%number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate() 
glueContext = GlueContext(sc) 
spark = glueContext.spark_session
job = Job(glueContext) 

Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 2
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 60
Session ID: 1c0acd5c-a377-469e-909b-e4fb45eadc49
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
--datalake-formats iceberg
--conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.type=glue --conf spark.sql.defaultCatalog=glue_catalog
Waiting for session 1c0acd5c-a377-469e-909b-e4fb45eadc49 to get into ready status...
Session 1c0acd5c-a377-469e-909b-e4fb45eadc49 has been created.



In [2]:
# MWAA에서 전달한 파라미터들을 받기
args = getResolvedOptions(sys.argv, [
    'JOB_NAME',           # 기본 파라미터 (항상 있음) \
    'yyyymm'      # --database_name=prod_db
]) 
yyyymm = args['yyyymm'] # 실제로는 Param으로 받아오기
# yyyymm = "202507"

s3_bucket = "s3://demo.nice.co.kr.datalake"
YYYY = yyyymm[:4]  # "2025"
MM = yyyymm[4:6]   # "07"
MM_1 = int(MM)

if MM_1 < 4:
    QTR = "1"
elif MM_1 < 7:
    QTR = "2"
elif MM_1 < 10:
    QTR = "3"
else:
    QTR = "4"

## 기본 데이터 적재 : getRaw_3.scala

In [3]:
spark.read \
    .parquet(f"{s3_bucket}/DI/CARD/KB_MEDI/RAW/GAIN_1/"+ YYYY + "/" + MM).cache() \
    .createOrReplaceTempView("KB_GAIN_1")

spark.read \
    .parquet(f"{s3_bucket}/DI/CARD/KB_MEDI/RAW/GAIN_2/"+ YYYY + "/" + MM).cache() \
    .createOrReplaceTempView("KB_GAIN_2")

spark.read \
    .parquet(f"{s3_bucket}/DI/CARD/KB_MEDI/LIST/YEAR=" + YYYY + "/QTR=" + QTR) \
    .createOrReplaceTempView("list")

spark.sql("""
    select a.*
      , b.area1_nm, b.area1_ratio
      , b.area2_nm, b.area2_ratio
      , b.area3_nm, b.area3_ratio
      , b.area4_nm, b.area4_ratio
      , b.area5_nm, b.area5_ratio
      , b.area6_nm, b.area6_ratio
      , b.area7_nm, b.area7_ratio
      , b.area8_nm, b.area8_ratio
      , b.area9_nm, b.area9_ratio
      , b.area10_nm, b.area10_ratio
    from KB_GAIN_1 a
      left join KB_GAIN_2 b
        on a.yyyymm = b.yyyymm
        and a.key = b.key
    """) \
    .createOrReplaceTempView("data")

In [4]:
%%sql
select count(1) from data

+--------+
|count(1)|
+--------+
|   88077|
+--------+


In [5]:
spark.sql("""
with data2 as (
  select  *
  , case
      when term_month / 12 < 1 then 1
      else 0
    end as term_1_under
  , case
      when term_month / 12 < 2 and term_month / 12 >= 1 then 1
      else 0
    end as term_1_2
  , case
      when term_month / 12 < 3 and term_month / 12 >= 2 then 1
      else 0
    end as term_2_3
  , case
      when term_month / 12 < 4 and term_month / 12 >= 3 then 1
      else 0
    end as term_3_4
  , case
      when term_month / 12 < 5 and term_month / 12 >= 4 then 1
      else 0
    end as term_4_5
  , case
      when term_month / 12 >= 5 then 1
      else 0
    end as term_5_over
  from data
)
select b.yyyymm,a.*
  , b.sale_cnt, b.sale_amt
  , b.cnt_m_20under
  , b.cnt_m_2024, b.cnt_m_2529
  , b.cnt_m_3034, b.cnt_m_3539
  , b.cnt_m_4044, b.cnt_m_4549
  , b.cnt_m_5054, b.cnt_m_5559
  , b.cnt_m_6064, b.cnt_m_6569
  , b.cnt_m_70over

  , b.cnt_w_20under
  , b.cnt_w_2024, b.cnt_w_2529
  , b.cnt_w_3034, b.cnt_w_3539
  , b.cnt_w_4044, b.cnt_w_4549
  , b.cnt_w_5054, b.cnt_w_5559
  , b.cnt_w_6064, b.cnt_w_6569
  , b.cnt_w_70over

  , b.sale_m_20under
  , b.sale_m_2024, b.sale_m_2529
  , b.sale_m_3034, b.sale_m_3539
  , b.sale_m_4044, b.sale_m_4549
  , b.sale_m_5054, b.sale_m_5559
  , b.sale_m_6064, b.sale_m_6569
  , b.sale_m_70over

  , b.sale_w_20under
  , b.sale_w_2024, b.sale_w_2529
  , b.sale_w_3034, b.sale_w_3539
  , b.sale_w_4044, b.sale_w_4549
  , b.sale_w_5054, b.sale_w_5559
  , b.sale_w_6064, b.sale_w_6569
  , b.sale_w_70over

  , b.cnt_mon, b.cnt_tue, b.cnt_wed, b.cnt_thu, b.cnt_fri, b.cnt_sat, b.cnt_sun
  , b.sale_mon, b.sale_tue, b.sale_wed, b.sale_thu, b.sale_fri, b.sale_sat, b.sale_sun

  , b.cnt_0003, b.cnt_0306, b.cnt_0609, b.cnt_0912, b.cnt_1215, b.cnt_1518, b.cnt_1821, b.cnt_2124
  , b.sale_0003, b.sale_0306, b.sale_0609, b.sale_0912, b.sale_1215, b.sale_1518, b.sale_1821, b.sale_2124

  , b.cnt_12m, b.sale_12m
  , b.cnt_6m, b.sale_6m
  , b.cnt_3m, b.sale_3m
  , b.cnt_1m, b.sale_1m
  , b.cnt_new, b.sale_new

  , b.term_month
  , b.term_1_under
  , b.term_1_2
  , b.term_2_3
  , b.term_3_4
  , b.term_4_5
  , b.term_5_over

  , b.area1_nm, b.area1_ratio
  , b.area2_nm, b.area2_ratio
  , b.area3_nm, b.area3_ratio
  , b.area4_nm, b.area4_ratio
  , b.area5_nm, b.area5_ratio
  , b.area6_nm, b.area6_ratio
  , b.area7_nm, b.area7_ratio
  , b.area8_nm, b.area8_ratio
  , b.area9_nm, b.area9_ratio
  , b.area10_nm, b.area10_ratio
from list a
  left join data2 b
    on a.key = b.key
where b.yyyymm is not null
""").cache().createOrReplaceTempView("raw_all")


In [6]:
%%sql
select count(1) from raw_all

+--------+
|count(1)|
+--------+
|   88073|
+--------+


## 병원데이터 : prep_hospital

In [7]:
# 영상의학과 제외

spark.sql("""
select *
from raw_all
where upjong_nm not rlike "영상" and upjong_nm not rlike "약국"
""").createOrReplaceTempView("raw_filtered")

spark.sql("""
with data as (
  select *
    , percent_rank() over(partition by upjong_nm, yyyymm order by sale_amt*1 desc) per_rnk_amt
  from raw_filtered
)
select *
  , case
    when per_rnk_amt >= 0.97 then 'low3p_amt'
    when per_rnk_amt < 0.97 and per_rnk_amt >= 0.95 then 'low5p_amt'
    when per_rnk_amt < 0.95 and per_rnk_amt >= 0.93 then 'low7p_amt'
    when per_rnk_amt < 0.93 and per_rnk_amt >= 0.90 then 'low10p_amt'
    else '100'
  end as gubun_amt
from data
order by sale_amt asc
""").createOrReplaceTempView("h_rank")

In [8]:
%%sql
select count(1) from h_rank

+--------+
|count(1)|
+--------+
|   64598|
+--------+


## target_revision

In [9]:
#업종별 보정대상 추출
spark.sql("""
select * from h_rank
where (upjong_nm = '가정의학과' or upjong_nm = '신경과' or upjong_nm = '정신건강의학과' or upjong_nm = '이비인후과')
and gubun_amt = 'low3p_amt'

union

select * from h_rank
where (upjong_nm = '내과' or upjong_nm = '정형외과')
and (gubun_amt = 'low3p_amt' or gubun_amt = 'low5p_amt')

union

select * from h_rank
where (upjong_nm = '마취통증의학과' or upjong_nm = '성형외과' or upjong_nm = '신경외과' or upjong_nm = '외과'  or upjong_nm = '재활의학과')
and (gubun_amt = 'low3p_amt' or gubun_amt = 'low5p_amt' or gubun_amt = 'low7p_amt')

union

select * from h_rank
where (upjong_nm = '비뇨기과' or upjong_nm = '산부인과' or upjong_nm = '소아과' or upjong_nm = '안과의원' or upjong_nm = '일반의원'  or upjong_nm = '피부과')
and (gubun_amt = 'low3p_amt' or gubun_amt = 'low5p_amt' or gubun_amt = 'low7p_amt' or gubun_amt = 'low10p_amt')
""").createOrReplaceTempView("m_target")

#보정값 추출
spark.sql("""
with data as (
  select yyyymm, upjong_nm, gubun_amt, min(bigint(sale_amt)) as sale_min
  from h_rank
  where gubun_amt = '100'
  group by 1,2,3
), data_1 as (
    select upjong_nm, sale_min from data
)
select distinct b.*
from data_1 a
left join raw_all b
on a.upjong_nm = b.upjong_nm
and a.sale_min = b.sale_amt
""").createOrReplaceTempView("m_value")

#보정대상 보정하기
spark.sql("""
select a.yyyymm, a.key, a.mega_cd, a.cty_cd, a.admi_cd, a.blk_cd_v3,a.upjong_nm, a.name as store_nm, a.addr, a.per_rnk_amt, a.gubun_amt
  ,b.sale_cnt,b.sale_amt,b.cnt_m_20under,b.cnt_m_2024,b.cnt_m_2529,b.cnt_m_3034,b.cnt_m_3539,b.cnt_m_4044,b.cnt_m_4549,b.cnt_m_5054,b.cnt_m_5559,b.cnt_m_6064,b.cnt_m_6569,b.cnt_m_70over,b.cnt_w_20under,b.cnt_w_2024,b.cnt_w_2529,b.cnt_w_3034,b.cnt_w_3539,b.cnt_w_4044,b.cnt_w_4549,b.cnt_w_5054,b.cnt_w_5559,b.cnt_w_6064,b.cnt_w_6569,b.cnt_w_70over,b.sale_m_20under,b.sale_m_2024,b.sale_m_2529,b.sale_m_3034,b.sale_m_3539,b.sale_m_4044,b.sale_m_4549,b.sale_m_5054,b.sale_m_5559,b.sale_m_6064,b.sale_m_6569,b.sale_m_70over,b.sale_w_20under,b.sale_w_2024,b.sale_w_2529,b.sale_w_3034,b.sale_w_3539,b.sale_w_4044,b.sale_w_4549,b.sale_w_5054,b.sale_w_5559,b.sale_w_6064,b.sale_w_6569,b.sale_w_70over,b.cnt_mon,b.cnt_tue,b.cnt_wed,b.cnt_thu,b.cnt_fri,b.cnt_sat,b.cnt_sun,b.sale_mon,b.sale_tue,b.sale_wed,b.sale_thu,b.sale_fri,b.sale_sat,b.sale_sun,b.cnt_0003,b.cnt_0306,b.cnt_0609,b.cnt_0912,b.cnt_1215,b.cnt_1518,b.cnt_1821,b.cnt_2124,b.sale_0003,b.sale_0306,b.sale_0609,b.sale_0912,b.sale_1215,b.sale_1518,b.sale_1821,b.sale_2124,b.cnt_12m,b.sale_12m,b.cnt_6m,b.sale_6m,b.cnt_3m,b.sale_3m,b.cnt_1m,b.sale_1m,b.cnt_new,b.sale_new
  ,a.term_month, a.term_1_under, a.term_1_2, a.term_2_3, a.term_3_4, a.term_4_5, a.term_5_over,a.area1_nm,a.area1_ratio,a.area2_nm,a.area2_ratio,a.area3_nm,a.area3_ratio,a.area4_nm,a.area4_ratio,a.area5_nm,a.area5_ratio,a.area6_nm,a.area6_ratio,a.area7_nm,a.area7_ratio,a.area8_nm,a.area8_ratio,a.area9_nm,a.area9_ratio,a.area10_nm,a.area10_ratio
from m_target a
  left join m_value b
  on a.yyyymm = b.yyyymm and a.upjong_nm = b.upjong_nm
order by upjong_nm
""").createOrReplaceTempView("modify_after")

In [10]:
%%sql
select count(1) from modify_after

+--------+
|count(1)|
+--------+
|    2476|
+--------+


## union_hospital

In [11]:
#보정대상 아닌 값
spark.sql("""
select * from h_rank
where (upjong_nm = '가정의학과' or upjong_nm = '신경과' or upjong_nm = '정신건강의학과' or upjong_nm = '이비인후과')
and gubun_amt != 'low3p_amt'

union

select * from h_rank
where (upjong_nm = '내과' or upjong_nm = '정형외과')
and !(gubun_amt = 'low3p_amt' or gubun_amt = 'low5p_amt')

union

select * from h_rank
where (upjong_nm = '마취통증의학과' or upjong_nm = '성형외과' or upjong_nm = '신경외과' or upjong_nm = '외과'  or upjong_nm = '재활의학과')
and !(gubun_amt = 'low3p_amt' or gubun_amt = 'low5p_amt' or gubun_amt = 'low7p_amt')

union

select * from h_rank
where (upjong_nm = '비뇨기과' or upjong_nm = '산부인과' or upjong_nm = '소아과' or upjong_nm = '안과의원' or upjong_nm = '일반의원'  or upjong_nm = '피부과')
and !(gubun_amt = 'low3p_amt' or gubun_amt = 'low5p_amt' or gubun_amt = 'low7p_amt' or gubun_amt = 'low10p_amt')
""").createOrReplaceTempView("normal")

#보정완료 데이터 + 보정대상 아닌 데이터 union
spark.sql("""
select yyyymm,key,mega_cd,cty_cd,admi_cd,blk_cd_v3,upjong_nm,name as store_nm,addr,per_rnk_amt,gubun_amt,sale_cnt,sale_amt,cnt_m_20under,cnt_m_2024,cnt_m_2529,cnt_m_3034,cnt_m_3539,cnt_m_4044,cnt_m_4549,cnt_m_5054,cnt_m_5559,cnt_m_6064,cnt_m_6569,cnt_m_70over,cnt_w_20under,cnt_w_2024,cnt_w_2529,cnt_w_3034,cnt_w_3539,cnt_w_4044,cnt_w_4549,cnt_w_5054,cnt_w_5559,cnt_w_6064,cnt_w_6569,cnt_w_70over,sale_m_20under,sale_m_2024,sale_m_2529,sale_m_3034,sale_m_3539,sale_m_4044,sale_m_4549,sale_m_5054,sale_m_5559,sale_m_6064,sale_m_6569,sale_m_70over,sale_w_20under,sale_w_2024,sale_w_2529,sale_w_3034,sale_w_3539,sale_w_4044,sale_w_4549,sale_w_5054,sale_w_5559,sale_w_6064,sale_w_6569,sale_w_70over,cnt_mon,cnt_tue,cnt_wed,cnt_thu,cnt_fri,cnt_sat,cnt_sun,sale_mon,sale_tue,sale_wed,sale_thu,sale_fri,sale_sat,sale_sun,cnt_0003,cnt_0306,cnt_0609,cnt_0912,cnt_1215,cnt_1518,cnt_1821,cnt_2124,sale_0003,sale_0306,sale_0609,sale_0912,sale_1215,sale_1518,sale_1821,sale_2124,cnt_12m,sale_12m,cnt_6m,sale_6m,cnt_3m,sale_3m,cnt_1m,sale_1m,cnt_new,sale_new,term_month,term_1_under,term_1_2,term_2_3,term_3_4,term_4_5,term_5_over,area1_nm,area1_ratio,area2_nm,area2_ratio,area3_nm,area3_ratio,area4_nm,area4_ratio,area5_nm,area5_ratio,area6_nm,area6_ratio,area7_nm,area7_ratio,area8_nm,area8_ratio,area9_nm,area9_ratio,area10_nm,area10_ratio
from normal

union

select *
from modify_after
""").createOrReplaceTempView("all")


#업종코드 붙이기
spark.sql("""
select
  case
    when upjong_nm = "가정의학과" then "S03010"
    when upjong_nm = "내과" then "S03001"
    when upjong_nm = "마취통증의학과" then "S04006"
    when upjong_nm = "비뇨기과" then "S03002"
    when upjong_nm = "산부인과" then "S03003"
    when upjong_nm = "성형외과" then "S04002"
    when upjong_nm = "소아과" then "S03004"
    when upjong_nm = "신경과" then "S04007"
    when upjong_nm = "신경외과" then "S04008"
    when upjong_nm = "안과의원" then "S04001"
    when upjong_nm = "외과" then "S03011"
    when upjong_nm = "이비인후과" then "S03005"
    when upjong_nm = "일반의원" then "S03009"
    when upjong_nm = "재활의학과" then "S04010"
    when upjong_nm = "정신건강의학과" then "S04011"
    when upjong_nm = "정형외과" then "S03008"
    when upjong_nm = "피부과" then "S04004"
  end as upjong_cd
, *
from all
""").createOrReplaceTempView("all_cd")

In [12]:
%%sql
select count(1) from all_cd

+--------+
|count(1)|
+--------+
|   34349|
+--------+


## multiplication
- 가중치 테이블을 만들어서 기존에 19번의 for 구분을 효율적으로 튜닝 최소 5배 이상 속도 개선 가능

In [13]:
import pandas as pd

multiple_idx = [2.37, 2.88, 1.48, 1.89, 1.71, 1.39, 3.06, 2.02, 1.43, 2.18, 1.53, 3.76, 2.47, 1.10, 3.74, 1.37, 1.01]

upjong_arr = ["S03010","S03001","S04006","S03002","S03003","S04002","S03004","S04007","S04008","S04001","S03011","S03005","S03009","S04010","S04011","S03008","S04004"]

# 가중치 테이블 생성
hospital_weight_table = pd.DataFrame({
    'multi_value': multiple_idx,
    'upjong_cd': upjong_arr
})

spark.createDataFrame(hospital_weight_table).createOrReplaceTempView("hospital_weight_table")
    
    

In [14]:
%%sql

select count(1) from hospital_weight_table

+--------+
|count(1)|
+--------+
|      17|
+--------+


In [15]:
spark.sql("""
select store_nm,key, a.upjong_cd, addr, yyyymm, mega_cd,cty_cd, admi_cd, blk_cd_v3

, cast((sale_cnt * multi_value) as decimal(20,0)) as sale_cnt, cast(sale_amt*multi_value as decimal(20,0)) as sale_amt

, cast(bigint(cnt_m_20under) * multi_value as decimal(20,0)) as cnt_m_20under, cast(bigint(cnt_m_2024)*multi_value as decimal(20,0)) as cnt_m_2024
, cast(bigint(cnt_m_2529)*multi_value as decimal(20,0)) as cnt_m_2529, cast(bigint(cnt_m_3034)*multi_value as decimal(20,0)) as cnt_m_3034
, cast(bigint(cnt_m_3539)*multi_value as decimal(20,0)) as cnt_m_3539, cast(bigint(cnt_m_4044)*multi_value as decimal(20,0)) as cnt_m_4044
, cast(bigint(cnt_m_4549)*multi_value as decimal(20,0)) as cnt_m_4549, cast(bigint(cnt_m_5054)*multi_value as decimal(20,0)) as cnt_m_5054
, cast(bigint(cnt_m_5559)*multi_value as decimal(20,0)) as cnt_m_5559, cast(bigint(cnt_m_6064)*multi_value as decimal(20,0)) as cnt_m_6064
, cast(bigint(cnt_m_6569)*multi_value as decimal(20,0)) as cnt_m_6569, cast(bigint(cnt_m_70over)*multi_value as decimal(20,0)) as cnt_m_70over
, cast(bigint(cnt_w_20under)*multi_value as decimal(20,0)) as cnt_w_20under, cast(bigint(cnt_w_2024)*multi_value as decimal(20,0)) as cnt_w_2024
, cast(bigint(cnt_w_2529)*multi_value as decimal(20,0)) as cnt_w_2529, cast(bigint(cnt_w_3034)*multi_value as decimal(20,0)) as cnt_w_3034
, cast(bigint(cnt_w_3539)*multi_value as decimal(20,0)) as cnt_w_3539, cast(bigint(cnt_w_4044)*multi_value as decimal(20,0)) as cnt_w_4044
, cast(bigint(cnt_w_4549)*multi_value as decimal(20,0)) as cnt_w_4549, cast(bigint(cnt_w_5054)*multi_value as decimal(20,0)) as cnt_w_5054
, cast(bigint(cnt_w_5559)*multi_value as decimal(20,0)) as cnt_w_5559, cast(bigint(cnt_w_6064)*multi_value as decimal(20,0)) as cnt_w_6064
, cast(bigint(cnt_w_6569)*multi_value as decimal(20,0)) as cnt_w_6569, cast(bigint(cnt_w_70over)*multi_value as decimal(20,0)) as cnt_w_70over

, cast(bigint(sale_m_20under)*multi_value as decimal(20,0)) as sale_m_20under, cast(bigint(sale_m_2024)*multi_value as decimal(20,0)) as sale_m_2024
, cast(bigint(sale_m_2529)*multi_value as decimal(20,0)) as sale_m_2529, cast(bigint(sale_m_3034)*multi_value as decimal(20,0)) as sale_m_3034
, cast(bigint(sale_m_3539)*multi_value as decimal(20,0)) as sale_m_3539, cast(bigint(sale_m_4044)*multi_value as decimal(20,0)) as sale_m_4044
, cast(bigint(sale_m_4549)*multi_value as decimal(20,0)) as sale_m_4549, cast(bigint(sale_m_5054)*multi_value as decimal(20,0)) as sale_m_5054
, cast(bigint(sale_m_5559)*multi_value as decimal(20,0)) as sale_m_5559, cast(bigint(sale_m_6064)*multi_value as decimal(20,0)) as sale_m_6064
, cast(bigint(sale_m_6569)*multi_value as decimal(20,0)) as sale_m_6569, cast(bigint(sale_m_70over)*multi_value as decimal(20,0)) as sale_m_70over
, cast(bigint(sale_w_20under)*multi_value as decimal(20,0)) as sale_w_20under, cast(bigint(sale_w_2024)*multi_value as decimal(20,0)) as sale_w_2024
, cast(bigint(sale_w_2529)*multi_value as decimal(20,0)) as sale_w_2529, cast(bigint(sale_w_3034)*multi_value as decimal(20,0)) as sale_w_3034
, cast(bigint(sale_w_3539)*multi_value as decimal(20,0)) as sale_w_3539, cast(bigint(sale_w_4044)*multi_value as decimal(20,0)) as sale_w_4044
, cast(bigint(sale_w_4549)*multi_value as decimal(20,0)) as sale_w_4549, cast(bigint(sale_w_5054)*multi_value as decimal(20,0)) as sale_w_5054
, cast(bigint(sale_w_5559)*multi_value as decimal(20,0)) as sale_w_5559, cast(bigint(sale_w_6064)*multi_value as decimal(20,0)) as sale_w_6064
, cast(bigint(sale_w_6569)*multi_value as decimal(20,0)) as sale_w_6569, cast(bigint(sale_w_70over)*multi_value as decimal(20,0)) as sale_w_70over

, cast(cnt_mon*multi_value as decimal(20,0)) as cnt_mon, cast(cnt_tue*multi_value as decimal(20,0)) as cnt_tue
, cast(cnt_wed*multi_value as decimal(20,0)) as cnt_wed, cast(cnt_thu*multi_value as decimal(20,0)) as cnt_thu
, cast(cnt_fri*multi_value as decimal(20,0)) as cnt_fri, cast(cnt_sat*multi_value as decimal(20,0)) as cnt_sat
, cast(cnt_sun*multi_value as decimal(20,0)) as cnt_sun, cast(sale_mon*multi_value as decimal(20,0)) as sale_mon
, cast(sale_tue*multi_value as decimal(20,0)) as sale_tue, cast(sale_wed*multi_value as decimal(20,0)) as sale_wed
, cast(sale_thu*multi_value as decimal(20,0)) as sale_thu, cast(sale_fri*multi_value as decimal(20,0)) as sale_fri
, cast(sale_sat*multi_value as decimal(20,0)) as sale_sat, cast(sale_sun*multi_value as decimal(20,0)) as sale_sun

, cast(cnt_0003*multi_value as decimal(20,0)) as cnt_0003, cast(cnt_0306*multi_value as decimal(20,0)) as cnt_0306
, cast(cnt_0609*multi_value as decimal(20,0)) as cnt_0609, cast(cnt_0912*multi_value as decimal(20,0)) as cnt_0912
, cast(cnt_1215*multi_value as decimal(20,0)) as cnt_1215, cast(cnt_1518*multi_value as decimal(20,0)) as cnt_1518
, cast(cnt_1821*multi_value as decimal(20,0)) as cnt_1821, cast(cnt_2124*multi_value as decimal(20,0)) as cnt_2124
, cast(sale_0003*multi_value as decimal(20,0)) as sale_0003, cast(sale_0306*multi_value as decimal(20,0)) as sale_0306
, cast(sale_0609*multi_value as decimal(20,0)) as sale_0609, cast(sale_0912*multi_value as decimal(20,0)) as sale_0912
, cast(sale_1215*multi_value as decimal(20,0)) as sale_1215, cast(sale_1518*multi_value as decimal(20,0)) as sale_1518
, cast(sale_1821*multi_value as decimal(20,0)) as sale_1821, cast(sale_2124*multi_value as decimal(20,0)) as sale_2124

, cast(cnt_12m*multi_value as decimal(20,0)) as cnt_12m, cast(sale_12m*multi_value as decimal(20,0)) as sale_12m
, cast(cnt_6m*multi_value as decimal(20,0)) as cnt_6m, cast(sale_6m*multi_value as decimal(20,0)) as sale_6m
, cast(cnt_3m*multi_value as decimal(20,0)) as cnt_3m, cast(sale_3m*multi_value as decimal(20,0)) as sale_3m
, cast(cnt_1m*multi_value as decimal(20,0)) as cnt_1m, cast(sale_1m*multi_value as decimal(20,0)) as sale_1m
, cast(cnt_new*multi_value as decimal(20,0)) as cnt_new, cast(sale_new*multi_value as decimal(20,0)) as sale_new

, term_month, term_1_under, term_1_2, term_2_3, term_3_4, term_4_5, term_5_over

, area1_nm, area1_ratio, area2_nm, area2_ratio, area3_nm, area3_ratio, area4_nm, area4_ratio, area5_nm, area5_ratio
, area6_nm, area6_ratio, area7_nm, area7_ratio, area8_nm, area8_ratio, area9_nm, area9_ratio, area10_nm, area10_ratio
from hospital_weight_table a
left outer join all_cd b
where a.upjong_cd = b.upjong_cd
""").createOrReplaceTempView("hospital_final")


In [16]:
%%sql
select count(1) from hospital_final

+--------+
|count(1)|
+--------+
|   34349|
+--------+


## 약국데이터 - prep_pharmacy

In [17]:
#하위 5% 구분
spark.sql("""
with data1 as (
  select *
  from raw_all where upjong_nm = "약국"
), data2 as (
  select *
    , percent_rank() over(partition by yyyymm order by sale_amt*1 desc) per_rnk_amt
  from data1
), data3 as (
  select *
    , case
        when per_rnk_amt >= 0.95 then 'low5p_amt'
        else '100'
      end gubun_amt
  from data2
)
select *
from data3
order by sale_amt asc
""").createOrReplaceTempView("drg_rank")

#보정대상 추출
spark.sql("""
select *
from drg_rank
where gubun_amt = "low5p_amt"
""").createOrReplaceTempView("pharmacy_m_target")

# .write
# .mode("overwrite")
# .parquet("/tmp/KB_MEDI/"+ yyyymm +"/PHARMACY/modify_target")

#보정값 추출
spark.sql("""
with min_value as (
  select yyyymm, gubun_amt, min(double(sale_amt)) as sale_min
  from drg_rank
  where gubun_amt = "100"
  group by yyyymm,gubun_amt
  order by yyyymm,gubun_amt
)
select *
from raw_all
where sale_amt = (select sale_min from min_value)
""").createOrReplaceTempView("pharmacy_m_value")
# .write
# .mode("overwrite")
# .parquet("/tmp/KB_MEDI/"+ yyyymm +"/PHARMACY/m_value")

# spark.read
# .parquet("/tmp/KB_MEDI/"+ yyyymm +"/PHARMACY/modify_target")
# .createOrReplaceTempView("target")

# spark.read
# .parquet("/tmp/KB_MEDI/"+ yyyymm +"/PHARMACY/m_value")
# .createOrReplaceTempView("m_value")

# 보정대상 보정값 붙이기
spark.sql("""
select a.yyyymm, a.key, a.mega_cd, a.cty_cd, a.admi_cd, a.blk_cd_v3,a.upjong_nm, a.name as store_nm, a.addr, a.per_rnk_amt, a.gubun_amt
  ,b.sale_cnt,b.sale_amt,b.cnt_m_20under,b.cnt_m_2024,b.cnt_m_2529,b.cnt_m_3034,b.cnt_m_3539,b.cnt_m_4044,b.cnt_m_4549,b.cnt_m_5054,b.cnt_m_5559,b.cnt_m_6064,b.cnt_m_6569,b.cnt_m_70over,b.cnt_w_20under,b.cnt_w_2024,b.cnt_w_2529,b.cnt_w_3034,b.cnt_w_3539,b.cnt_w_4044,b.cnt_w_4549,b.cnt_w_5054,b.cnt_w_5559,b.cnt_w_6064,b.cnt_w_6569,b.cnt_w_70over,b.sale_m_20under,b.sale_m_2024,b.sale_m_2529,b.sale_m_3034,b.sale_m_3539,b.sale_m_4044,b.sale_m_4549,b.sale_m_5054,b.sale_m_5559,b.sale_m_6064,b.sale_m_6569,b.sale_m_70over,b.sale_w_20under,b.sale_w_2024,b.sale_w_2529,b.sale_w_3034,b.sale_w_3539,b.sale_w_4044,b.sale_w_4549,b.sale_w_5054,b.sale_w_5559,b.sale_w_6064,b.sale_w_6569,b.sale_w_70over,b.cnt_mon,b.cnt_tue,b.cnt_wed,b.cnt_thu,b.cnt_fri,b.cnt_sat,b.cnt_sun,b.sale_mon,b.sale_tue,b.sale_wed,b.sale_thu,b.sale_fri,b.sale_sat,b.sale_sun,b.cnt_0003,b.cnt_0306,b.cnt_0609,b.cnt_0912,b.cnt_1215,b.cnt_1518,b.cnt_1821,b.cnt_2124,b.sale_0003,b.sale_0306,b.sale_0609,b.sale_0912,b.sale_1215,b.sale_1518,b.sale_1821,b.sale_2124,b.cnt_12m,b.sale_12m,b.cnt_6m,b.sale_6m,b.cnt_3m,b.sale_3m,b.cnt_1m,b.sale_1m,b.cnt_new,b.sale_new
  ,a.term_month, a.term_1_under, a.term_1_2, a.term_2_3, a.term_3_4, a.term_4_5, a.term_5_over,a.area1_nm,a.area1_ratio,a.area2_nm,a.area2_ratio,a.area3_nm,a.area3_ratio,a.area4_nm,a.area4_ratio,a.area5_nm,a.area5_ratio,a.area6_nm,a.area6_ratio,a.area7_nm,a.area7_ratio,a.area8_nm,a.area8_ratio,a.area9_nm,a.area9_ratio,a.area10_nm,a.area10_ratio
from pharmacy_m_target a
  left join pharmacy_m_value b
  on a.yyyymm = b.yyyymm
""").createOrReplaceTempView("pharmacy_modify_after")

# 보정대상 + 보정비대상
spark.sql("""
select yyyymm, key,blk_cd_v3, mega_cd, cty_cd, admi_cd, "S02004" as upjong3_cd,upjong_nm as upjong3_nm, name as store_nm, addr, per_rnk_amt, gubun_amt,
  sale_cnt, sale_amt, cnt_m_20under, cnt_m_2024, cnt_m_2529, cnt_m_3034, cnt_m_3539, cnt_m_4044, cnt_m_4549, cnt_m_5054, cnt_m_5559, cnt_m_6064, cnt_m_6569, cnt_m_70over, cnt_w_20under, cnt_w_2024, cnt_w_2529, cnt_w_3034, cnt_w_3539, cnt_w_4044, cnt_w_4549, cnt_w_5054, cnt_w_5559, cnt_w_6064, cnt_w_6569, cnt_w_70over, sale_m_20under, sale_m_2024, sale_m_2529, sale_m_3034, sale_m_3539, sale_m_4044, sale_m_4549, sale_m_5054, sale_m_5559, sale_m_6064, sale_m_6569, sale_m_70over, sale_w_20under, sale_w_2024, sale_w_2529, sale_w_3034, sale_w_3539, sale_w_4044, sale_w_4549, sale_w_5054, sale_w_5559, sale_w_6064, sale_w_6569, sale_w_70over, cnt_mon, cnt_tue, cnt_wed, cnt_thu, cnt_fri, cnt_sat, cnt_sun, sale_mon, sale_tue, sale_wed, sale_thu, sale_fri, sale_sat, sale_sun, cnt_0003, cnt_0306, cnt_0609, cnt_0912, cnt_1215, cnt_1518, cnt_1821, cnt_2124, sale_0003, sale_0306, sale_0609, sale_0912, sale_1215, sale_1518, sale_1821, sale_2124, cnt_12m, sale_12m, cnt_6m, sale_6m, cnt_3m, sale_3m, cnt_1m, sale_1m, cnt_new, sale_new, term_month, term_1_under, term_1_2, term_2_3, term_3_4, term_4_5, term_5_over, area1_nm, area1_ratio, area2_nm, area2_ratio, area3_nm, area3_ratio, area4_nm, area4_ratio, area5_nm, area5_ratio, area6_nm, area6_ratio, area7_nm, area7_ratio, area8_nm, area8_ratio, area9_nm, area9_ratio, area10_nm, area10_ratio
from drg_rank
where gubun_amt != "low5p_amt"

union

select yyyymm, key,blk_cd_v3, mega_cd, cty_cd, admi_cd, "S02004" as upjong3_cd,upjong_nm as upjong3_nm, store_nm, addr, per_rnk_amt, gubun_amt,
  sale_cnt, sale_amt, cnt_m_20under, cnt_m_2024, cnt_m_2529, cnt_m_3034, cnt_m_3539, cnt_m_4044, cnt_m_4549, cnt_m_5054, cnt_m_5559, cnt_m_6064, cnt_m_6569, cnt_m_70over, cnt_w_20under, cnt_w_2024, cnt_w_2529, cnt_w_3034, cnt_w_3539, cnt_w_4044, cnt_w_4549, cnt_w_5054, cnt_w_5559, cnt_w_6064, cnt_w_6569, cnt_w_70over, sale_m_20under, sale_m_2024, sale_m_2529, sale_m_3034, sale_m_3539, sale_m_4044, sale_m_4549, sale_m_5054, sale_m_5559, sale_m_6064, sale_m_6569, sale_m_70over, sale_w_20under, sale_w_2024, sale_w_2529, sale_w_3034, sale_w_3539, sale_w_4044, sale_w_4549, sale_w_5054, sale_w_5559, sale_w_6064, sale_w_6569, sale_w_70over, cnt_mon, cnt_tue, cnt_wed, cnt_thu, cnt_fri, cnt_sat, cnt_sun, sale_mon, sale_tue, sale_wed, sale_thu, sale_fri, sale_sat, sale_sun, cnt_0003, cnt_0306, cnt_0609, cnt_0912, cnt_1215, cnt_1518, cnt_1821, cnt_2124, sale_0003, sale_0306, sale_0609, sale_0912, sale_1215, sale_1518, sale_1821, sale_2124, cnt_12m, sale_12m, cnt_6m, sale_6m, cnt_3m, sale_3m, cnt_1m, sale_1m, cnt_new, sale_new, term_month, term_1_under, term_1_2, term_2_3, term_3_4, term_4_5, term_5_over, area1_nm, area1_ratio, area2_nm, area2_ratio, area3_nm, area3_ratio, area4_nm, area4_ratio, area5_nm, area5_ratio, area6_nm, area6_ratio, area7_nm, area7_ratio, area8_nm, area8_ratio, area9_nm, area9_ratio, area10_nm, area10_ratio
from pharmacy_modify_after
""").createOrReplaceTempView("pharmacy_prep_data")

# .write
# .mode("overwrite")
# .parquet("/tmp/KB_MEDI/"+ yyyymm +"/PHARMACY/union_all")

In [18]:
%%sql

select count(1) from pharmacy_prep_data

+--------+
|count(1)|
+--------+
|   23475|
+--------+


## multiplication_p
- 가중치 테이블을 만들어서 기존에 229번의 for 구분을 효율적으로 튜닝 최소 20배 이상 속도 개선 가능

In [19]:
import pandas as pd

sale_idx = [0.342957262, 0.33769736, 0.392112409, 0.331816916, 0.44000807, 0.405501254, 0.310310828, 0.329176474, 0.325888702, 0.443308983
, 0.391407639, 0.313328055, 0.363644866, 0.330054365, 0.349425543, 0.401383614, 0.368991603, 0.347446809, 0.444336123, 0.399500802
, 0.324715964, 0.32232946, 0.336662014, 0.349311047, 0.373218914, 0.617156638, 0.580234197, 0.608281193, 0.58057875, 0.547266106, 0.559762055
, 0.631231039, 0.59437928, 0.579746279, 0.543699053, 0.642341014, 0.615680106, 0.599263974, 0.598280667, 0.644957047, 0.587432689, 0.781296962
, 0.742444559, 0.936022693, 0.926941533, 0.82699464, 0.7296425, 0.771034352, 0.704021702, 0.786690369, 0.676845447, 0.696612063, 0.73466872
, 0.791535665, 0.714915063, 0.674788225, 0.75056331, 1.079956261, 1, 0.595352996, 0.519059377, 0.557947106, 0.476209184, 0.526585201, 0.550772684
, 0.555671735, 0.478793273, 0.455975486, 0.62021103, 0.702183373, 0.635176255, 0.695970464, 0.687879646, 0.715739866, 0.538626048, 0.452880173
, 0.389854342, 0.409863114, 0.417108425, 0.388588624, 0.362592235, 0.571488541, 0.455660962, 0.478791411, 0.371612043, 0.515842394, 0.386079382
, 0.410846987, 0.549458487, 0.441620074, 0.443104597, 0.419963057, 0.427546278, 0.435324387, 0.4695268, 0.804417091, 0.697196394, 0.436262868
, 0.519048948, 0.542711634, 0.428270142, 0.529922917, 0.620225085, 0.515102092, 0.656753889, 0.598641565, 1.036674735, 1.220990358, 1.166556516
, 1.941908369, 1.212489448, 2.979132192, 0.825139037, 2.624693918, 1.49629262, 0.69036742, 2.162681086, 1.001248664, 0.86514477, 0.956003069
, 1.107358832, 1.257195787, 1.215032161, 0.556771619, 1.208502544, 2.138559357, 1.541000436, 2.006559698, 1.671070415, 0.944441866, 1.163991544
, 1.514366186, 0.915525538, 0.645662171, 0.70146817, 0.842831262, 0.796890766, 1.034903086, 1.541749257, 1.796422963, 1.507036769, 1.831525989
, 0.826542423, 1.195957481, 1.594289686, 1.023726381, 2.464821726, 0.788801278, 1.32167246, 2.410195415, 1.371875738, 1.153179586, 1.710814708
, 1.214340239, 2.225505251, 0.635600753, 0.879321602, 0.930752587, 0.854845877, 0.692509668, 0.871783595, 0.585880692, 1.065157327, 0.855350397
, 0.688835258, 0.777387593, 1.338439164, 1.208267712, 1.21471885, 1.720626702, 1.233413266, 1.458855938, 0.700874031, 0.643407267, 1.640012992
, 0.863733419, 0.831932635, 0.701702676, 0.783446809, 0.586906039, 0.827146077, 0.592527177, 0.971358112, 0.518742637, 0.667061568, 1.901877261
, 1.402857098, 1.511740127, 1.612162345, 1.328942606, 1.565634547, 1.537266706, 1.741491456, 0.841120045, 1.650068681, 1.115275674, 1.099719976
, 0.898697987, 0.841915318, 1.037662785, 0.625588417, 1.231085614, 0.772550908, 0.786847432, 0.894128033, 1.104397535, 1.749400179, 1.457758429
, 1.761830301, 0.862827722, 1.148645285, 0.96570122, 0.773521066, 1.23314926, 1.039486799, 0.762659236, 0.907463348, 1.109482714, 1.002944577
, 1.166218141, 1.749750043, 0.94528374, 0.98426932, 2.958846624, 1.802792186, 1.465618606, 2.036002216, 1.673775725, 1.244220894 ]

cnt_idx = [0.304575942, 0.301845579, 0.379165271, 0.300454078, 0.432022941, 0.36602906, 0.313041832, 0.33960164, 0.269313451, 0.217589577, 0.468834026, 0.379522916, 0.351861997, 0.327832394, 0.323045752, 0.252738654, 0.374100719, 0.341401459, 0.423335859, 0.361532239, 0.296959022, 0.303032687, 0.295800724, 0.327055974, 0.343336652, 0.570029709, 0.563661946, 0.577897838
, 0.572584311, 0.525519001, 0.533790569, 0.590594263, 0.558381547, 0.53420134, 0.512542909, 0.608295592, 0.539125718, 0.566859743, 0.575462974, 0.575873197, 0.557679217, 0.710840237, 0.67341591, 0.832831523, 0.806697225, 0.773428668, 0.68463243, 0.720038923, 0.630798941, 0.795057178, 0.691288863, 0.715936475, 0.718095408, 0.80230288, 0.718877593, 0.673720038
, 0.739670231, 0.997081454, 1, 0.545414252, 0.49129079, 0.489680225, 0.468487143, 0.505176822, 0.50902578, 0.548999758, 0.432464846, 0.415481206, 0.570403159, 0.669054492, 0.581443581, 0.658565158, 0.627345794, 0.660739857, 0.505328759, 0.440339179, 0.413344724, 0.419736379, 0.410383058, 0.409856424, 0.374141107, 0.553840157, 0.470723247, 0.467411787, 0.396953754
, 0.519005629, 0.422935738, 0.433125769, 0.515230983, 0.410648098, 0.482373331, 0.416680375, 0.406920369, 0.426931965, 0.455197314, 0.716246333, 0.636617636, 0.417638268, 0.487458829, 0.536331113, 0.452205836, 0.482928309, 0.582823605, 0.464116546, 0.597938144, 0.534415096, 0.959375674, 1.065965888, 1.061432429, 1.59137189, 1.246767751, 2.550706316, 0.729746003, 1.688879457
, 1.330540616, 0.672384526, 1.504645137, 0.984153335, 0.800788479, 0.909122638, 1.02147569, 1.029520745, 1.062585478, 0.520802545, 1.033169705, 1.901860841, 1.392575784, 1.771766606, 1.43390772, 0.921844744, 1.075885784, 1.33684143, 0.764967867, 0.586341738, 0.639012433, 0.690737183, 0.703240508, 0.925799139, 1.18308732, 1.369168811, 1.307776091, 1.442427879, 0.740385651
, 1.220591661, 1.28420191, 0.845894579, 1.461450053, 0.688252086, 1.170991824, 1.805212565, 1.098485083, 1.11996362, 1.277984666, 1.048081593, 1.929082628, 0.593622873, 0.829703795, 0.861616734, 0.729595532, 0.674086801, 0.77236261, 0.523676217, 0.897111453, 0.732929216, 0.619957374, 0.792788069, 0.928199272, 0.903666147, 1.029297761, 1.381956501, 1.00023015, 1.21477984
, 0.700100691, 0.54926883, 1.235820896, 0.783072992, 0.604704463, 0.64102698, 0.662038588, 0.601766022, 0.734742018, 0.535663997, 0.879308304, 0.490052666, 0.650866829, 1.295778435, 1.2121942, 1.327197747, 1.29632335, 1.08218708, 1.226263313, 1.411775771, 1.498201073, 0.783987383, 1.327546089, 0.932554902, 0.939099567, 0.806052358, 0.776953643, 0.897912389, 0.614976454
, 1.21713033, 0.697457692, 0.835846868, 0.77886919, 0.936846151, 1.467695158, 1.137215292, 1.406938776, 0.649439969, 0.856033119, 0.792152796, 0.693316469, 0.9791261, 0.77610173, 0.700514122, 0.774210968, 1.028407489, 0.926497602, 1.044529878, 1.52553472, 0.777403809, 0.947798652, 2.138056754, 1.629756306, 1.140028541, 1.754899627, 1.529364492, 0.997155812
]

cty_arr = [1111, 1114, 1117, 1120, 1121, 1123, 1126, 1129, 1130, 1132, 1135, 1138, 1141, 1144, 1147, 1150, 1153, 1154, 1156, 1159, 1162, 1165, 1168, 1171, 1174, 2611, 2614, 2617, 2620, 2623, 2626, 2629, 2632, 2635, 2638, 2641, 2644, 2647, 2650, 2653, 2671, 2711, 2714, 2717, 2720, 2723, 2726, 2729, 2771, 2811, 2814, 2817, 2818, 2820, 2823, 2824, 2826, 2871, 2872, 2911, 2914, 2915, 2917
, 2920, 3011, 3014, 3017, 3020, 3023, 3111, 3114, 3117, 3120, 3171, 3611, 4111, 4113, 4115, 4117, 4119, 4121, 4122, 4125, 4127, 4128, 4129, 4131, 4136, 4137, 4139, 4141, 4143, 4145, 4146, 4148, 4150, 4155, 4157, 4159, 4161, 4163, 4165, 4167, 4180, 4182, 4183, 4311, 4313, 4315, 4372, 4373, 4374, 4375, 4376, 4377, 4378, 4380, 4413, 4415, 4418, 4420, 4421, 4423, 4425, 4427, 4471
, 4476, 4477, 4479, 4480, 4481, 4482, 4611, 4613, 4615, 4617, 4623, 4671, 4672, 4673, 4677, 4678, 4679, 4680, 4681, 4682, 4683, 4684, 4686, 4687, 4688, 4689, 4690, 4691, 2772, 4711, 4713, 4715, 4717, 4719, 4721, 4723, 4725, 4728, 4729, 4773, 4775, 4776, 4777, 4782, 4783, 4784, 4785, 4790, 4792, 4793, 4794, 4812, 4817, 4822, 4824, 4825, 4827, 4831, 4833, 4872, 4873, 4874, 4882
, 4884, 4885, 4886, 4887, 4888, 4889, 5011, 5013, 5111, 5113, 5115, 5117, 5119, 5121, 5123, 5172, 5173, 5175, 5176, 5177, 5178, 5179, 5180, 5181, 5182, 5183, 5211, 5213, 5214, 5218, 5219, 5221, 5271, 5272, 5273, 5274, 5275, 5277, 5279, 5280]

# 가중치 테이블 생성
pharmacy_weight_table = pd.DataFrame({
    'cty_cd': cty_arr,
    'sale_multivalue': sale_idx,
    'cnt_multivalue': cnt_idx
})

spark.createDataFrame(pharmacy_weight_table).createOrReplaceTempView("pharmacy_weight_table")

In [20]:
%%sql
select * from pharmacy_weight_table

+------+---------------+--------------+
|cty_cd|sale_multivalue|cnt_multivalue|
+------+---------------+--------------+
|  1111|    0.342957262|   0.304575942|
|  1114|     0.33769736|   0.301845579|
|  1117|    0.392112409|   0.379165271|
|  1120|    0.331816916|   0.300454078|
|  1121|     0.44000807|   0.432022941|
|  1123|    0.405501254|    0.36602906|
|  1126|    0.310310828|   0.313041832|
|  1129|    0.329176474|    0.33960164|
|  1130|    0.325888702|   0.269313451|
|  1132|    0.443308983|   0.217589577|
|  1135|    0.391407639|   0.468834026|
|  1138|    0.313328055|   0.379522916|
|  1141|    0.363644866|   0.351861997|
|  1144|    0.330054365|   0.327832394|
|  1147|    0.349425543|   0.323045752|
|  1150|    0.401383614|   0.252738654|
|  1153|    0.368991603|   0.374100719|
|  1154|    0.347446809|   0.341401459|
|  1156|    0.444336123|   0.423335859|
|  1159|    0.399500802|   0.361532239|
+------+---------------+--------------+
only showing top 20 rows


In [22]:
spark.sql("""
  select store_nm,key, upjong3_cd, upjong3_nm, addr, yyyymm, mega_cd, a.cty_cd, admi_cd, blk_cd_v3

    , cast((sale_cnt * cnt_multivalue) as decimal(20,0)) as sale_cnt, cast(sale_amt*sale_multivalue as decimal(20,0)) as sale_amt

    , cast(bigint(cnt_m_20under) * cnt_multivalue as decimal(20,0)) as cnt_m_20under, cast(bigint(cnt_m_2024)*cnt_multivalue as decimal(20,0)) as cnt_m_2024
    , cast(bigint(cnt_m_2529)*cnt_multivalue as decimal(20,0)) as cnt_m_2529, cast(bigint(cnt_m_3034)*cnt_multivalue as decimal(20,0)) as cnt_m_3034
    , cast(bigint(cnt_m_3539)*cnt_multivalue as decimal(20,0)) as cnt_m_3539, cast(bigint(cnt_m_4044)*cnt_multivalue as decimal(20,0)) as cnt_m_4044
    , cast(bigint(cnt_m_4549)*cnt_multivalue as decimal(20,0)) as cnt_m_4549, cast(bigint(cnt_m_5054)*cnt_multivalue as decimal(20,0)) as cnt_m_5054
    , cast(bigint(cnt_m_5559)*cnt_multivalue as decimal(20,0)) as cnt_m_5559, cast(bigint(cnt_m_6064)*cnt_multivalue as decimal(20,0)) as cnt_m_6064
    , cast(bigint(cnt_m_6569)*cnt_multivalue as decimal(20,0)) as cnt_m_6569, cast(bigint(cnt_m_70over)*cnt_multivalue as decimal(20,0)) as cnt_m_70over
    , cast(bigint(cnt_w_20under)*cnt_multivalue as decimal(20,0)) as cnt_w_20under, cast(bigint(cnt_w_2024)*cnt_multivalue as decimal(20,0)) as cnt_w_2024
    , cast(bigint(cnt_w_2529)*cnt_multivalue as decimal(20,0)) as cnt_w_2529, cast(bigint(cnt_w_3034)*cnt_multivalue as decimal(20,0)) as cnt_w_3034
    , cast(bigint(cnt_w_3539)*cnt_multivalue as decimal(20,0)) as cnt_w_3539, cast(bigint(cnt_w_4044)*cnt_multivalue as decimal(20,0)) as cnt_w_4044
    , cast(bigint(cnt_w_4549)*cnt_multivalue as decimal(20,0)) as cnt_w_4549, cast(bigint(cnt_w_5054)*cnt_multivalue as decimal(20,0)) as cnt_w_5054
    , cast(bigint(cnt_w_5559)*cnt_multivalue as decimal(20,0)) as cnt_w_5559, cast(bigint(cnt_w_6064)*cnt_multivalue as decimal(20,0)) as cnt_w_6064
    , cast(bigint(cnt_w_6569)*cnt_multivalue as decimal(20,0)) as cnt_w_6569, cast(bigint(cnt_w_70over)*cnt_multivalue as decimal(20,0)) as cnt_w_70over

    , cast(bigint(sale_m_20under)*sale_multivalue as decimal(20,0)) as sale_m_20under, cast(bigint(sale_m_2024)*sale_multivalue as decimal(20,0)) as sale_m_2024
    , cast(bigint(sale_m_2529)*sale_multivalue as decimal(20,0)) as sale_m_2529, cast(bigint(sale_m_3034)*sale_multivalue as decimal(20,0)) as sale_m_3034
    , cast(bigint(sale_m_3539)*sale_multivalue as decimal(20,0)) as sale_m_3539, cast(bigint(sale_m_4044)*sale_multivalue as decimal(20,0)) as sale_m_4044
    , cast(bigint(sale_m_4549)*sale_multivalue as decimal(20,0)) as sale_m_4549, cast(bigint(sale_m_5054)*sale_multivalue as decimal(20,0)) as sale_m_5054
    , cast(bigint(sale_m_5559)*sale_multivalue as decimal(20,0)) as sale_m_5559, cast(bigint(sale_m_6064)*sale_multivalue as decimal(20,0)) as sale_m_6064
    , cast(bigint(sale_m_6569)*sale_multivalue as decimal(20,0)) as sale_m_6569, cast(bigint(sale_m_70over)*sale_multivalue as decimal(20,0)) as sale_m_70over
    , cast(bigint(sale_w_20under)*sale_multivalue as decimal(20,0)) as sale_w_20under, cast(bigint(sale_w_2024)*sale_multivalue as decimal(20,0)) as sale_w_2024
    , cast(bigint(sale_w_2529)*sale_multivalue as decimal(20,0)) as sale_w_2529, cast(bigint(sale_w_3034)*sale_multivalue as decimal(20,0)) as sale_w_3034
    , cast(bigint(sale_w_3539)*sale_multivalue as decimal(20,0)) as sale_w_3539, cast(bigint(sale_w_4044)*sale_multivalue as decimal(20,0)) as sale_w_4044
    , cast(bigint(sale_w_4549)*sale_multivalue as decimal(20,0)) as sale_w_4549, cast(bigint(sale_w_5054)*sale_multivalue as decimal(20,0)) as sale_w_5054
    , cast(bigint(sale_w_5559)*sale_multivalue as decimal(20,0)) as sale_w_5559, cast(bigint(sale_w_6064)*sale_multivalue as decimal(20,0)) as sale_w_6064
    , cast(bigint(sale_w_6569)*sale_multivalue as decimal(20,0)) as sale_w_6569, cast(bigint(sale_w_70over)*sale_multivalue as decimal(20,0)) as sale_w_70over

    , cast(cnt_mon*cnt_multivalue as decimal(20,0)) as cnt_mon, cast(cnt_tue*cnt_multivalue as decimal(20,0)) as cnt_tue
    , cast(cnt_wed*cnt_multivalue as decimal(20,0)) as cnt_wed, cast(cnt_thu*cnt_multivalue as decimal(20,0)) as cnt_thu
    , cast(cnt_fri*cnt_multivalue as decimal(20,0)) as cnt_fri, cast(cnt_sat*cnt_multivalue as decimal(20,0)) as cnt_sat
    , cast(cnt_sun*cnt_multivalue as decimal(20,0)) as cnt_sun, cast(sale_mon*sale_multivalue as decimal(20,0)) as sale_mon
    , cast(sale_tue*sale_multivalue as decimal(20,0)) as sale_tue, cast(sale_wed*sale_multivalue as decimal(20,0)) as sale_wed
    , cast(sale_thu*sale_multivalue as decimal(20,0)) as sale_thu, cast(sale_fri*sale_multivalue as decimal(20,0)) as sale_fri
    , cast(sale_sat*sale_multivalue as decimal(20,0)) as sale_sat, cast(sale_sun*sale_multivalue as decimal(20,0)) as sale_sun

    , cast(cnt_0003*cnt_multivalue as decimal(20,0)) as cnt_0003, cast(cnt_0306*cnt_multivalue as decimal(20,0)) as cnt_0306
    , cast(cnt_0609*cnt_multivalue as decimal(20,0)) as cnt_0609, cast(cnt_0912*cnt_multivalue as decimal(20,0)) as cnt_0912
    , cast(cnt_1215*cnt_multivalue as decimal(20,0)) as cnt_1215, cast(cnt_1518*cnt_multivalue as decimal(20,0)) as cnt_1518
    , cast(cnt_1821*cnt_multivalue as decimal(20,0)) as cnt_1821, cast(cnt_2124*cnt_multivalue as decimal(20,0)) as cnt_2124
    , cast(sale_0003*sale_multivalue as decimal(20,0)) as sale_0003, cast(sale_0306*sale_multivalue as decimal(20,0)) as sale_0306
    , cast(sale_0609*sale_multivalue as decimal(20,0)) as sale_0609, cast(sale_0912*sale_multivalue as decimal(20,0)) as sale_0912
    , cast(sale_1215*sale_multivalue as decimal(20,0)) as sale_1215, cast(sale_1518*sale_multivalue as decimal(20,0)) as sale_1518
    , cast(sale_1821*sale_multivalue as decimal(20,0)) as sale_1821, cast(sale_2124*sale_multivalue as decimal(20,0)) as sale_2124

    , cast(cnt_12m*cnt_multivalue as decimal(20,0)) as cnt_12m, cast(sale_12m*sale_multivalue as decimal(20,0)) as sale_12m
    , cast(cnt_6m*cnt_multivalue as decimal(20,0)) as cnt_6m, cast(sale_6m*sale_multivalue as decimal(20,0)) as sale_6m
    , cast(cnt_3m*cnt_multivalue as decimal(20,0)) as cnt_3m, cast(sale_3m*sale_multivalue as decimal(20,0)) as sale_3m
    , cast(cnt_1m*cnt_multivalue as decimal(20,0)) as cnt_1m, cast(sale_1m*sale_multivalue as decimal(20,0)) as sale_1m
    , cast(cnt_new*cnt_multivalue as decimal(20,0)) as cnt_new, cast(sale_new*sale_multivalue as decimal(20,0)) as sale_new

    , term_month, term_1_under, term_1_2, term_2_3, term_3_4, term_4_5, term_5_over

    , area1_nm, area1_ratio, area2_nm, area2_ratio, area3_nm, area3_ratio, area4_nm, area4_ratio, area5_nm, area5_ratio
    , area6_nm, area6_ratio, area7_nm, area7_ratio, area8_nm, area8_ratio, area9_nm, area9_ratio, area10_nm, area10_ratio
  from pharmacy_weight_table a
  left outer join pharmacy_prep_data b
  where a.cty_cd = b.cty_cd
""").createOrReplaceTempView("pharmacy_final") 

# .coalesce(1)
# .write
# .mode("overwrite")
# .parquet("/tmp/KB_MEDI/"+yyyymm+"/PHARMACY/multiplication/T_" + cty_cd)

In [23]:
%%sql

select count(1) from pharmacy_final

+--------+
|count(1)|
+--------+
|   23475|
+--------+


## 치과데이터 추출 : prep_dentist


In [24]:
# <<< 이상치 제거를 위한 매출 등급 매기기 - 치과 이상치 추가 (2024.04.08) >>>

spark.sql("""
-- 치과 선택
with data1 as (
  select *
  from raw_all
  where upjong_nm = "치과"
)
, data2 as (
  select *
    , percent_rank() over(partition by upjong_nm, yyyymm order by sale_amt*1 desc) per_rnk_amt -- 매출rank별
  from data1
)
, data3 as (
select *
  , case
      when per_rnk_amt <= 0.001 then 'high0.1p_amt' -- 상위 0.1%
      when per_rnk_amt >= 0.90 then 'low10p_amt' -- 하위 10%
      else '100'
    end as gubun_amt
  from data2
)
select *
from data3
""").createOrReplaceTempView("tb_dentist")

# -- 상위 0.1%
spark.sql("""
select *
from tb_dentist
where 1=1
and upjong_nm = "치과"
and gubun_amt = 'high0.1p_amt'
""").createOrReplaceTempView("tb_dentist_high")

# -- 하위 10% (2024.04.01 치과 추가)
spark.sql("""
select *
from tb_dentist
where gubun_amt = 'low10p_amt'
""").createOrReplaceTempView("tb_dentist_low")

spark.sql("""
select *
from tb_dentist
where gubun_amt = '100'
""").createOrReplaceTempView("tb_dentist_normal")

# 보정값 추출(high,low)
# 하위
spark.sql("""
with data as (
  select yyyymm, upjong_nm, gubun_amt, min(bigint(sale_amt)) as sale_min
  from tb_dentist
  where gubun_amt = '100'
  group by 1,2,3
)
select *
from raw_all
where upjong_nm = "치과"
and sale_amt in (select sale_min from data)
""").createOrReplaceTempView("dentist_m_value_low")

# 상위
spark.sql("""
with data as (
  select yyyymm, upjong_nm, gubun_amt, max(bigint(sale_amt)) as sale_min
  from tb_dentist
  where gubun_amt = '100'
  group by 1,2,3
)
select *
from raw_all
where upjong_nm = "치과"
and sale_amt in (select sale_min from data)
""").createOrReplaceTempView("dentist_m_value_high")

#보정대상 보정하기
# 상위 0.1%
spark.sql("""
select a.yyyymm, a.key, a.mega_cd, a.cty_cd, a.admi_cd, a.blk_cd_v3,a.upjong_nm, a.name as store_nm, a.addr, a.per_rnk_amt, a.gubun_amt
  , b.sale_cnt,b.sale_amt,b.cnt_m_20under,b.cnt_m_2024,b.cnt_m_2529,b.cnt_m_3034,b.cnt_m_3539,b.cnt_m_4044,b.cnt_m_4549,b.cnt_m_5054,b.cnt_m_5559,b.cnt_m_6064,b.cnt_m_6569,b.cnt_m_70over,b.cnt_w_20under,b.cnt_w_2024,b.cnt_w_2529,b.cnt_w_3034,b.cnt_w_3539,b.cnt_w_4044,b.cnt_w_4549,b.cnt_w_5054,b.cnt_w_5559,b.cnt_w_6064,b.cnt_w_6569,b.cnt_w_70over,b.sale_m_20under,b.sale_m_2024,b.sale_m_2529,b.sale_m_3034,b.sale_m_3539,b.sale_m_4044,b.sale_m_4549,b.sale_m_5054,b.sale_m_5559,b.sale_m_6064,b.sale_m_6569,b.sale_m_70over,b.sale_w_20under,b.sale_w_2024,b.sale_w_2529,b.sale_w_3034,b.sale_w_3539,b.sale_w_4044,b.sale_w_4549,b.sale_w_5054,b.sale_w_5559,b.sale_w_6064,b.sale_w_6569,b.sale_w_70over,b.cnt_mon,b.cnt_tue,b.cnt_wed,b.cnt_thu,b.cnt_fri,b.cnt_sat,b.cnt_sun,b.sale_mon,b.sale_tue,b.sale_wed,b.sale_thu,b.sale_fri,b.sale_sat,b.sale_sun,b.cnt_0003,b.cnt_0306,b.cnt_0609,b.cnt_0912,b.cnt_1215,b.cnt_1518,b.cnt_1821,b.cnt_2124,b.sale_0003,b.sale_0306,b.sale_0609,b.sale_0912,b.sale_1215,b.sale_1518,b.sale_1821,b.sale_2124,b.cnt_12m,b.sale_12m,b.cnt_6m,b.sale_6m,b.cnt_3m,b.sale_3m,b.cnt_1m,b.sale_1m,b.cnt_new,b.sale_new
  , a.term_month, a.term_1_under, a.term_1_2, a.term_2_3, a.term_3_4, a.term_4_5, a.term_5_over,a.area1_nm,a.area1_ratio,a.area2_nm,a.area2_ratio,a.area3_nm,a.area3_ratio,a.area4_nm,a.area4_ratio,a.area5_nm,a.area5_ratio,a.area6_nm,a.area6_ratio,a.area7_nm,a.area7_ratio,a.area8_nm,a.area8_ratio,a.area9_nm,a.area9_ratio,a.area10_nm,a.area10_ratio
from tb_dentist_high a
  left join dentist_m_value_high b
    on a.yyyymm = b.yyyymm and a.upjong_nm = b.upjong_nm
order by upjong_nm
""").createOrReplaceTempView("dentist_m_after_high")

# .coalesce(5)
# .write
# .mode("overwrite")
# .parquet("/tmp/KB_MEDI/" + yyyymm + "/DENTIST/PREP_HIGH")

#보정대상 보정하기
# 하위 10%
spark.sql("""
select a.yyyymm, a.key, a.mega_cd, a.cty_cd, a.admi_cd, a.blk_cd_v3,a.upjong_nm, a.name as store_nm, a.addr, a.per_rnk_amt, a.gubun_amt
  , b.sale_cnt,b.sale_amt,b.cnt_m_20under,b.cnt_m_2024,b.cnt_m_2529,b.cnt_m_3034,b.cnt_m_3539,b.cnt_m_4044,b.cnt_m_4549,b.cnt_m_5054,b.cnt_m_5559,b.cnt_m_6064,b.cnt_m_6569,b.cnt_m_70over,b.cnt_w_20under,b.cnt_w_2024,b.cnt_w_2529,b.cnt_w_3034,b.cnt_w_3539,b.cnt_w_4044,b.cnt_w_4549,b.cnt_w_5054,b.cnt_w_5559,b.cnt_w_6064,b.cnt_w_6569,b.cnt_w_70over,b.sale_m_20under,b.sale_m_2024,b.sale_m_2529,b.sale_m_3034,b.sale_m_3539,b.sale_m_4044,b.sale_m_4549,b.sale_m_5054,b.sale_m_5559,b.sale_m_6064,b.sale_m_6569,b.sale_m_70over,b.sale_w_20under,b.sale_w_2024,b.sale_w_2529,b.sale_w_3034,b.sale_w_3539,b.sale_w_4044,b.sale_w_4549,b.sale_w_5054,b.sale_w_5559,b.sale_w_6064,b.sale_w_6569,b.sale_w_70over,b.cnt_mon,b.cnt_tue,b.cnt_wed,b.cnt_thu,b.cnt_fri,b.cnt_sat,b.cnt_sun,b.sale_mon,b.sale_tue,b.sale_wed,b.sale_thu,b.sale_fri,b.sale_sat,b.sale_sun,b.cnt_0003,b.cnt_0306,b.cnt_0609,b.cnt_0912,b.cnt_1215,b.cnt_1518,b.cnt_1821,b.cnt_2124,b.sale_0003,b.sale_0306,b.sale_0609,b.sale_0912,b.sale_1215,b.sale_1518,b.sale_1821,b.sale_2124,b.cnt_12m,b.sale_12m,b.cnt_6m,b.sale_6m,b.cnt_3m,b.sale_3m,b.cnt_1m,b.sale_1m,b.cnt_new,b.sale_new
  , a.term_month, a.term_1_under, a.term_1_2, a.term_2_3, a.term_3_4, a.term_4_5, a.term_5_over,a.area1_nm,a.area1_ratio,a.area2_nm,a.area2_ratio,a.area3_nm,a.area3_ratio,a.area4_nm,a.area4_ratio,a.area5_nm,a.area5_ratio,a.area6_nm,a.area6_ratio,a.area7_nm,a.area7_ratio,a.area8_nm,a.area8_ratio,a.area9_nm,a.area9_ratio,a.area10_nm,a.area10_ratio
from tb_dentist_low a
  left join dentist_m_value_low b
    on a.yyyymm = b.yyyymm and a.upjong_nm = b.upjong_nm
order by upjong_nm
""").createOrReplaceTempView("dentist_m_after_low")

# .coalesce(5)
# .write
# .mode("overwrite")
# .parquet("/tmp/KB_MEDI/" + yyyymm + "/DENTIST/PREP_LOW")

# spark.read
# .parquet("/tmp/KB_MEDI/" + yyyymm + "/DENTIST/PREP_HIGH")
# .createOrReplaceTempView("m_after_high")

# spark.read
# .parquet("/tmp/KB_MEDI/" + yyyymm + "/DENTIST/PREP_LOW")
# .createOrReplaceTempView("m_after_low")

#상위 + 중간 + 하위 모두 union
spark.sql("""
select *
from dentist_m_after_high

union

select yyyymm,key,mega_cd,cty_cd,admi_cd,blk_cd_v3,upjong_nm,name as store_nm,addr,per_rnk_amt,gubun_amt,sale_cnt,sale_amt,cnt_m_20under,cnt_m_2024,cnt_m_2529,cnt_m_3034,cnt_m_3539,cnt_m_4044,cnt_m_4549,cnt_m_5054,cnt_m_5559,cnt_m_6064,cnt_m_6569,cnt_m_70over,cnt_w_20under,cnt_w_2024,cnt_w_2529,cnt_w_3034,cnt_w_3539,cnt_w_4044,cnt_w_4549,cnt_w_5054,cnt_w_5559,cnt_w_6064,cnt_w_6569,cnt_w_70over,sale_m_20under,sale_m_2024,sale_m_2529,sale_m_3034,sale_m_3539,sale_m_4044,sale_m_4549,sale_m_5054,sale_m_5559,sale_m_6064,sale_m_6569,sale_m_70over,sale_w_20under,sale_w_2024,sale_w_2529,sale_w_3034,sale_w_3539,sale_w_4044,sale_w_4549,sale_w_5054,sale_w_5559,sale_w_6064,sale_w_6569,sale_w_70over,cnt_mon,cnt_tue,cnt_wed,cnt_thu,cnt_fri,cnt_sat,cnt_sun,sale_mon,sale_tue,sale_wed,sale_thu,sale_fri,sale_sat,sale_sun,cnt_0003,cnt_0306,cnt_0609,cnt_0912,cnt_1215,cnt_1518,cnt_1821,cnt_2124,sale_0003,sale_0306,sale_0609,sale_0912,sale_1215,sale_1518,sale_1821,sale_2124,cnt_12m,sale_12m,cnt_6m,sale_6m,cnt_3m,sale_3m,cnt_1m,sale_1m,cnt_new,sale_new,term_month,term_1_under,term_1_2,term_2_3,term_3_4,term_4_5,term_5_over,area1_nm,area1_ratio,area2_nm,area2_ratio,area3_nm,area3_ratio,area4_nm,area4_ratio,area5_nm,area5_ratio,area6_nm,area6_ratio,area7_nm,area7_ratio,area8_nm,area8_ratio,area9_nm,area9_ratio,area10_nm,area10_ratio
from tb_dentist_normal

union

select *
from dentist_m_after_low
""").createOrReplaceTempView("union_dentist")

spark.sql("""
select yyyymm,key,store_nm,addr,"S04003" as upjong3_cd,mega_cd,cty_cd,admi_cd,blk_cd_v3,sale_cnt,sale_amt,cnt_m_20under,cnt_m_2024,cnt_m_2529,cnt_m_3034,cnt_m_3539,cnt_m_4044,cnt_m_4549,cnt_m_5054,cnt_m_5559,cnt_m_6064,cnt_m_6569,cnt_m_70over,cnt_w_20under,cnt_w_2024,cnt_w_2529,cnt_w_3034,cnt_w_3539,cnt_w_4044,cnt_w_4549,cnt_w_5054,cnt_w_5559,cnt_w_6064,cnt_w_6569,cnt_w_70over,sale_m_20under,sale_m_2024,sale_m_2529,sale_m_3034,sale_m_3539,sale_m_4044,sale_m_4549,sale_m_5054,sale_m_5559,sale_m_6064,sale_m_6569,sale_m_70over,sale_w_20under,sale_w_2024,sale_w_2529,sale_w_3034,sale_w_3539,sale_w_4044,sale_w_4549,sale_w_5054,sale_w_5559,sale_w_6064,sale_w_6569,sale_w_70over,cnt_mon,cnt_tue,cnt_wed,cnt_thu,cnt_fri,cnt_sat,cnt_sun,sale_mon,sale_tue,sale_wed,sale_thu,sale_fri,sale_sat,sale_sun,cnt_0003,cnt_0306,cnt_0609,cnt_0912,cnt_1215,cnt_1518,cnt_1821,cnt_2124,sale_0003,sale_0306,sale_0609,sale_0912,sale_1215,sale_1518,sale_1821,sale_2124,cnt_12m,sale_12m,cnt_6m,sale_6m,cnt_3m,sale_3m,cnt_1m,sale_1m,cnt_new,sale_new,term_month,term_1_under,term_1_2,term_2_3,term_3_4,term_4_5,term_5_over,area1_nm,area1_ratio,area2_nm,area2_ratio,area3_nm,area3_ratio,area4_nm,area4_ratio,area5_nm,area5_ratio,area6_nm,area6_ratio,area7_nm,area7_ratio,area8_nm,area8_ratio,area9_nm,area9_ratio,area10_nm,area10_ratio
from union_dentist
""").createOrReplaceTempView("dentist_final")
# .write
# .mode("overwrite")
# .parquet("/tmp/KB_MEDI/" + yyyymm + "/DENTIST/raw_final")

In [34]:
%%sql
select count(1) from dentist_final

+--------+
|count(1)|
+--------+
|   17154|
+--------+


In [26]:
## 마지막 데이터 병합 : merge_raw
#의원+약국 RAW 생성
final_df = spark.sql("""
with data as (
    select yyyymm,key,store_nm,addr,upjong3_cd,mega_cd,cty_cd,admi_cd,blk_cd_v3,sale_cnt,sale_amt,cnt_m_20under,cnt_m_2024,cnt_m_2529,cnt_m_3034,cnt_m_3539,cnt_m_4044,cnt_m_4549,cnt_m_5054,cnt_m_5559,cnt_m_6064,cnt_m_6569,cnt_m_70over,cnt_w_20under,cnt_w_2024,cnt_w_2529,cnt_w_3034,cnt_w_3539,cnt_w_4044,cnt_w_4549,cnt_w_5054,cnt_w_5559,cnt_w_6064,cnt_w_6569,cnt_w_70over,sale_m_20under,sale_m_2024,sale_m_2529,sale_m_3034,sale_m_3539,sale_m_4044,sale_m_4549,sale_m_5054,sale_m_5559,sale_m_6064,sale_m_6569,sale_m_70over,sale_w_20under,sale_w_2024,sale_w_2529,sale_w_3034,sale_w_3539,sale_w_4044,sale_w_4549,sale_w_5054,sale_w_5559,sale_w_6064,sale_w_6569,sale_w_70over,cnt_mon,cnt_tue,cnt_wed,cnt_thu,cnt_fri,cnt_sat,cnt_sun,sale_mon,sale_tue,sale_wed,sale_thu,sale_fri,sale_sat,sale_sun,cnt_0003,cnt_0306,cnt_0609,cnt_0912,cnt_1215,cnt_1518,cnt_1821,cnt_2124,sale_0003,sale_0306,sale_0609,sale_0912,sale_1215,sale_1518,sale_1821,sale_2124,cnt_12m,sale_12m,cnt_6m,sale_6m,cnt_3m,sale_3m,cnt_1m,sale_1m,cnt_new,sale_new,term_month,term_1_under,term_1_2,term_2_3,term_3_4,term_4_5,term_5_over,area1_nm,area1_ratio,area2_nm,area2_ratio,area3_nm,area3_ratio,area4_nm,area4_ratio,area5_nm,area5_ratio,area6_nm,area6_ratio,area7_nm,area7_ratio,area8_nm,area8_ratio,area9_nm,area9_ratio,area10_nm,area10_ratio
    from pharmacy_final

    union

    select yyyymm,key,store_nm,addr,upjong_cd as upjong3_cd,mega_cd,cty_cd,admi_cd,blk_cd_v3,sale_cnt,sale_amt,cnt_m_20under,cnt_m_2024,cnt_m_2529,cnt_m_3034,cnt_m_3539,cnt_m_4044,cnt_m_4549,cnt_m_5054,cnt_m_5559,cnt_m_6064,cnt_m_6569,cnt_m_70over,cnt_w_20under,cnt_w_2024,cnt_w_2529,cnt_w_3034,cnt_w_3539,cnt_w_4044,cnt_w_4549,cnt_w_5054,cnt_w_5559,cnt_w_6064,cnt_w_6569,cnt_w_70over,sale_m_20under,sale_m_2024,sale_m_2529,sale_m_3034,sale_m_3539,sale_m_4044,sale_m_4549,sale_m_5054,sale_m_5559,sale_m_6064,sale_m_6569,sale_m_70over,sale_w_20under,sale_w_2024,sale_w_2529,sale_w_3034,sale_w_3539,sale_w_4044,sale_w_4549,sale_w_5054,sale_w_5559,sale_w_6064,sale_w_6569,sale_w_70over,cnt_mon,cnt_tue,cnt_wed,cnt_thu,cnt_fri,cnt_sat,cnt_sun,sale_mon,sale_tue,sale_wed,sale_thu,sale_fri,sale_sat,sale_sun,cnt_0003,cnt_0306,cnt_0609,cnt_0912,cnt_1215,cnt_1518,cnt_1821,cnt_2124,sale_0003,sale_0306,sale_0609,sale_0912,sale_1215,sale_1518,sale_1821,sale_2124,cnt_12m,sale_12m,cnt_6m,sale_6m,cnt_3m,sale_3m,cnt_1m,sale_1m,cnt_new,sale_new,term_month,term_1_under,term_1_2,term_2_3,term_3_4,term_4_5,term_5_over,area1_nm,area1_ratio,area2_nm,area2_ratio,area3_nm,area3_ratio,area4_nm,area4_ratio,area5_nm,area5_ratio,area6_nm,area6_ratio,area7_nm,area7_ratio,area8_nm,area8_ratio,area9_nm,area9_ratio,area10_nm,area10_ratio
    from hospital_final

    union

    select  yyyymm,key,store_nm,addr, upjong3_cd,mega_cd,cty_cd,admi_cd,blk_cd_v3,sale_cnt,sale_amt,cnt_m_20under,cnt_m_2024,cnt_m_2529,cnt_m_3034,cnt_m_3539,cnt_m_4044,cnt_m_4549,cnt_m_5054,cnt_m_5559,cnt_m_6064,cnt_m_6569,cnt_m_70over,cnt_w_20under,cnt_w_2024,cnt_w_2529,cnt_w_3034,cnt_w_3539,cnt_w_4044,cnt_w_4549,cnt_w_5054,cnt_w_5559,cnt_w_6064,cnt_w_6569,cnt_w_70over,sale_m_20under,sale_m_2024,sale_m_2529,sale_m_3034,sale_m_3539,sale_m_4044,sale_m_4549,sale_m_5054,sale_m_5559,sale_m_6064,sale_m_6569,sale_m_70over,sale_w_20under,sale_w_2024,sale_w_2529,sale_w_3034,sale_w_3539,sale_w_4044,sale_w_4549,sale_w_5054,sale_w_5559,sale_w_6064,sale_w_6569,sale_w_70over,cnt_mon,cnt_tue,cnt_wed,cnt_thu,cnt_fri,cnt_sat,cnt_sun,sale_mon,sale_tue,sale_wed,sale_thu,sale_fri,sale_sat,sale_sun,cnt_0003,cnt_0306,cnt_0609,cnt_0912,cnt_1215,cnt_1518,cnt_1821,cnt_2124,sale_0003,sale_0306,sale_0609,sale_0912,sale_1215,sale_1518,sale_1821,sale_2124,cnt_12m,sale_12m,cnt_6m,sale_6m,cnt_3m,sale_3m,cnt_1m,sale_1m,cnt_new,sale_new,term_month,term_1_under,term_1_2,term_2_3,term_3_4,term_4_5,term_5_over,area1_nm,area1_ratio,area2_nm,area2_ratio,area3_nm,area3_ratio,area4_nm,area4_ratio,area5_nm,area5_ratio,area6_nm,area6_ratio,area7_nm,area7_ratio,area8_nm,area8_ratio,area9_nm,area9_ratio,area10_nm,area10_ratio
    from dentist_final
)
select * from data
where blk_cd_v3 is not null and mega_cd is not null and cty_cd is not null and admi_cd is not null
""").createOrReplaceTempView("all_final")
# .write
# .mode("overwrite")
# .parquet("/DATA/DW/CARD/KB_MEDI/TBKBMEDI_STORE_TMP/" + yyyymm)

# val PH_nn = spark.read.parquet("/tmp/KB_MEDI/"+yyyymm+"/PHARMACY/multiplication/*").count()
# val HOS_nn = spark.read.parquet("/tmp/KB_MEDI/" + yyyymm + "/HOSPITAL/raw_last").count()
# val DEN_nn = spark.read.parquet("/tmp/KB_MEDI/" + yyyymm + "/DENTIST/raw_final").count()
# val ALL_nn = spark.read.parquet("/DATA/DW/CARD/KB_MEDI/TBKBMEDI_STORE_TMP/" + yyyymm).count()
# val ALL_uni_nn = PH_nn + HOS_nn + DEN_nn
# val cnt_chk = ALL_nn <= ALL_uni_nn
# val drop_nn = ALL_uni_nn - ALL_nn

# println("PHARMACY_FINAL_CNT: "+ PH_nn)
# println("HOSPITAL_FINAL_CNT: "+ HOS_nn)
# println("DENTIST_FINAL_CNT: "+DEN_nn)
# println("ALL_CNT: " + ALL_nn)
# println("CNT_CHK: "+ cnt_chk + " (DROP_CNT: " + drop_nn + ")")

In [35]:
%%sql
select distinct yyyymm from all_final

+------+
|yyyymm|
+------+
|202507|
+------+


In [27]:
%%sql

select 'pharmacy_final' as type, count(1) cnt from pharmacy_final
union all
select 'hospital_final', count(1) cnt from hospital_final
union all
select 'dentist_final', count(1) cnt from dentist_final
union all
select 'all_final', count(1) cnt from all_final


+--------------+-----+
|          type|  cnt|
+--------------+-----+
|pharmacy_final|23475|
|hospital_final|34349|
| dentist_final|17154|
|     all_final|74772|
+--------------+-----+


## 최종결과를 iceberg 테이블로 덮어쓰기
- yyyymm 컬럼 파티션명이랑 동일하여 drop하고 insert

In [38]:
spark.sql(f"""
INSERT OVERWRITE nice_database.tbkbcard_medi_store 
PARTITION (yyyymm = '{yyyymm}')
SELECT 
key,store_nm,addr,upjong3_cd,mega_cd,cty_cd,admi_cd,blk_cd_v3,sale_cnt,sale_amt,cnt_m_20under,cnt_m_2024,cnt_m_2529,cnt_m_3034,cnt_m_3539,cnt_m_4044,cnt_m_4549,cnt_m_5054,cnt_m_5559,cnt_m_6064,cnt_m_6569,cnt_m_70over,cnt_w_20under,cnt_w_2024,cnt_w_2529,cnt_w_3034,cnt_w_3539,cnt_w_4044,cnt_w_4549,cnt_w_5054,cnt_w_5559,cnt_w_6064,cnt_w_6569,cnt_w_70over,sale_m_20under,sale_m_2024,sale_m_2529,sale_m_3034,sale_m_3539,sale_m_4044,sale_m_4549,sale_m_5054,sale_m_5559,sale_m_6064,sale_m_6569,sale_m_70over,sale_w_20under,sale_w_2024,sale_w_2529,sale_w_3034,sale_w_3539,sale_w_4044,sale_w_4549,sale_w_5054,sale_w_5559,sale_w_6064,sale_w_6569,sale_w_70over,cnt_mon,cnt_tue,cnt_wed,cnt_thu,cnt_fri,cnt_sat,cnt_sun,sale_mon,sale_tue,sale_wed,sale_thu,sale_fri,sale_sat,sale_sun,cnt_0003,cnt_0306,cnt_0609,cnt_0912,cnt_1215,cnt_1518,cnt_1821,cnt_2124,sale_0003,sale_0306,sale_0609,sale_0912,sale_1215,sale_1518,sale_1821,sale_2124,cnt_12m,sale_12m,cnt_6m,sale_6m,cnt_3m,sale_3m,cnt_1m,sale_1m,cnt_new,sale_new,term_month,term_1_under,term_1_2,term_2_3,term_3_4,term_4_5,term_5_over,area1_nm,area1_ratio,area2_nm,area2_ratio,area3_nm,area3_ratio,area4_nm,area4_ratio,area5_nm,area5_ratio,area6_nm,area6_ratio,area7_nm,area7_ratio,area8_nm,area8_ratio,area9_nm,area9_ratio,area10_nm,area10_ratio
FROM all_final
""")

DataFrame[]


In [39]:
spark.sql(f"""
select count(1) cnt from nice_database.tbkbcard_medi_store 
where yyyymm = '{yyyymm}'
""").show()

+-----+
|  cnt|
+-----+
|74772|
+-----+
